# **Lab 3: Approximation**
**Matteus Berg**

2024-02-06

# **Abstract**

In this lab report, the L2 projection method is implemented. Linear shape functions are used for the implemented projection method. The code is then tested for its accuracy and convergence rate.

The results show that the projection method has quadratic convergence rate when approximating non-linear functions.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

The objective of this lab report is to run and test the L2 projection method. The purpose of L2 projection is to approximate a function f in a set of nodes $\{x_i\}^N_{i=1}$. The approximation for f is denoted as $P_Nf(x)$. The L2 projection is defined in the course book on page 200 as

$ P_Nf(x) = \sum_{j=1}^N \alpha_j\phi_j(x) $

Where $\{ \phi \}_{i=1}^N$ is a collection of basis vectors that span $L^2([a,b])$, a hilbert space. And $\{ \alpha \}_{i=1}^N$ a collection of coordinates in $V_N$. After a basis is chosen, the main task is to compute the coordinates $\alpha \in R^N$ by restating the above equation as a matrix equation

$ \sum_{j=1}^N \alpha_j(\phi_j,\phi_i) = (f,\phi_i) \hspace{30pt} \forall i = 1,...,N $

In the above equation, $\{\alpha_j\}_{j=1}^{N}$ is the vector of elements that we want to determine. The $L^2([a,b])$ inner product is defined as

$(f,g) = (f,g)_{L^2([a,b])} = \int_a^b f(x)g(x) dx$

In order to solve the matrix equation we need to determine $A$ and $b$. For this we need to compute all the inner products that $A$ and $b$ is made up of. The so-called "assembly algorithm" is implemented for this. See page 203 in the book. Instead of iterating over each interval $I_k = [x_{k-1}, x_k]$ several times to get each value of the matrix and vector, we instead iterate over each interval once and calculate all integrals in that interval.

To get a feel for the matrix equation it can help looking at its structure for N = 3:

$
A\alpha = b <=>
\begin{bmatrix}
\phi_1(x_1) & \phi_2(x_1) & \phi_3(x_1) \\
\phi_1(x_2) & \phi_2(x_2) & \phi_3(x_2) \\
\phi_1(x_3) & \phi_2(x_3) & \phi_3(x_3) \\
\end{bmatrix}
*
\begin{bmatrix}
\alpha_1 \\
\alpha_2 \\
\alpha_3 \\
\end{bmatrix}
=
\begin{bmatrix}
f(x_1) \\
f(x_2) \\
f(x_3) \\
\end{bmatrix}
$

With $\phi$ being a nodal basis, we get that

$
A\alpha = b <=>
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
*
\begin{bmatrix}
\alpha_1 \\
\alpha_2 \\
\alpha_3 \\
\end{bmatrix}
=
\begin{bmatrix}
f(x_1) \\
f(x_2) \\
f(x_3) \\
\end{bmatrix}
=> \alpha_j = f(x_j)
$

# **Method**

The implemented solution can be seen below. The "L_2_projection" function calls the "assemble_system" function to determine $A$ and $b$. It then solves the equation and returns the resulting function values for the inputed node values.

The main loop in "assemble_system" loops over every interval $I_k = [x_{k-1}, x_k]$ and calculates a local sub vector $b_k$ and a sub matrix $a_k$ and then adds them to their global counterparts $b$ and $A$. All sub matrices and sub vectors have the size 2x2 and 2 respectively, except for the two edge intervals where the sizes are 1x1 and 1.

The function "get_no_local_shape_functions" is a helper function used by "assembly_system" to determine whether it is currently iterating over one of the two edge intervals. Those two intervals only have one shape function instead of two.

"lambda_function" is a helper method used to determine which of the two lambda functions to use depending on the value of "j".

In [2]:


# number of integration steps to take for each integration done for
# for each element of Ak matrix and bk vector
steps = 100

def L2_projection(fx, nodes: np.ndarray):
  mplus1 = nodes.size+2
  A = np.zeros((mplus1, mplus1))
  b = np.zeros(mplus1)
  (A, b) = assemble_system(fx, A, b, nodes)
  x = np.linalg.solve(A, b)

  # don't return the two edge values for x
  remove_edges = [0, x.size - 1]

  return np.delete(x, remove_edges)

# page 203 in the book
def assemble_system(f, A: np.ndarray, b: np.ndarray, nodes: np.ndarray):
  numElements = b.size # add the two edge nodes
  for k in range(1, numElements):
    q = get_no_local_shape_functions(k, numElements)
    loc2glob = get_local_to_global_map(k, numElements)
    b_k = np.zeros(q)
    a_k = np.zeros((q, q))
    for i in range(0, q):
      b_k[i] = integrate_vector(f, k, i, numElements, nodes)
      for j in range(0, q):
        a_k[i,j] = integrate_matrix(k, i, j, numElements, nodes)

    # add_to_global_vector
    for i in range(0, q):
      #print("k: ", k, "; loc2glob(i): ", loc2glob(i))
      b[loc2glob(i)] += b_k[i]

    # add_to_global_matrix
    for i in range(0, q):
      for j in range(0, q):
        A[loc2glob(i), loc2glob(j)] += a_k[i, j]

  return (A, b)

def get_no_local_shape_functions(k, numElements):
  if k == 1 or k == numElements-1:
    return 1
  else:
    return 2

def get_local_to_global_map(k, numElements):
  if k != 1 and k != numElements-1:
    loc2glob = lambda j : k-1+j
  # special case for k = 0 and k = m+1
  else:
    if k == 1:
      loc2glob = lambda j : 0
    elif k == numElements-1:
      loc2glob = lambda j : numElements-1
  return loc2glob

def integrate_vector(f, k: int, i: int, numElements: int, nodes: np.ndarray):
  # special cases for edge nodes
  if k == 1:
    hk = nodes[1] - nodes[0]
    xk = nodes[0]
    shapeFunction = lambda x : (xk - x)/hk
    currentX = xk - hk
  elif k == numElements-1:
    hk = nodes[numElements-3] - nodes[numElements-4]
    xkminus1 = nodes[numElements-3]
    shapeFunction = lambda x : (x - xkminus1)/hk
    currentX = xkminus1
  else: # case for internal noddes
    xk = nodes[(k-1)]
    xkminus1 = nodes[(k-1)-1]
    hk = xk - xkminus1
    shapeFunction = lambda_function(k, i, xk, xkminus1)
    currentX = xkminus1

  # store values from f(x)*lambda_{k,i}(x) product
  values = np.zeros(steps)

  # calculate values
  for i in range(0, steps):
    values[i] = f(currentX)*shapeFunction(currentX)
    currentX = currentX + hk/steps

  result = np.trapz(values, dx=hk/steps)

  return result


def integrate_matrix(k: int, i: int, j: int, numElements: int, nodes: np.ndarray):
  if k == 1:
    hk = nodes[1] - nodes[0]
    xk = nodes[0]
    shapeFunction1 = lambda x : (xk - x)/hk
    shapeFunction2 = shapeFunction1
    currentX = xk - hk
  elif k == numElements-1:
    hk = nodes[numElements-3] - nodes[numElements-4]
    xkminus1 = nodes[numElements-3]
    shapeFunction1 = lambda x : (x - xkminus1)/hk
    shapeFunction2 = shapeFunction1
    currentX = xkminus1
  else:
    xk = nodes[(k-1)]
    xkminus1 = nodes[(k-1)-1]
    hk = xk - xkminus1
    shapeFunction1 = lambda_function(k, j, xk, xkminus1)
    shapeFunction2 = lambda_function(k, i, xk, xkminus1)
    currentX = xkminus1

  # store values from f(x)*lambda_{k,i}(x) product
  values = np.zeros(steps)

  # calculate values
  for i in range(0, steps):
    values[i] = shapeFunction1(currentX)*shapeFunction2(currentX)
    currentX = currentX + hk/steps

  result = np.trapz(values, dx=hk/steps)

  return result


# create local shape function and return it
def lambda_function(k: int, j: int, xk: int, xkminus1: int):
  hk = xk - xkminus1
  if j == 0:
    return lambda x : (xk - x)/hk
  elif j == 1:
    return lambda x : (x - xkminus1)/hk

f = lambda x : x*x
x = np.array([0, 1])

y = L2_projection(f, x)

print(y)

var = 2










[-0.16333333  0.82666667]


# **Results**

The below results consist of testing the L2 projection method using different functions f, nodes x and integration step density

Accuracy test 1:
* function f = 2 * x + 1
* steps = 100

| nodes  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
|---|---|---|---|---||---|---|---|---|---|
| output | 3 | 5 | 7 | 9 | 11 | 13 | 15 | 17 | 19 | 21 |

* nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
* received output = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

Accuracy test 2:
* function f = x*x
* steps = 100

| nodes  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
|---|---|---|---|---||---|---|---|---|---|
| output | 0.83 | 3.83 | 8.83 |15.83 | 24.83 | 35.83 | 48.83 | 63.83 | 80.83 | 99.82 |

Convergance test 1:
* function f = x*x
* nodes [-1, 0, 1]

| steps  | -1 | 0 | 1 |
|---|---|---|---|
| 2 | 0.916 | -0.083 | 0.583 |
| 4 | 0.848 | -0.160 | 0.735 |
| 8 | 0.840 | -0.175 | 0.785 |
| 16 | 0.837 | -0.174 | 0.809 |

Convergance test 2:
* function f = x*x
* steps = 100
* domain [0, 1]

| # nodes in domain | 0 | 1 |
|---|---|---|
| 2 | 0.1633 | 0.8267 |
| 4 | -0.0180 | 0.9799 |
| 8 | -0.0045 | 0.9946 |

# **Discussion**

The first accuracy test show that the L2_projection is 100\% accurate for linear functions regardless of distance between nodes. This is to be expected.

The first convergence test shows that after a certain point, increasing the amount of steps for the integration no longer has an effect on the convergence rate. If the distance between nodes is small enough it won't matter integrating with shorter step sizes.

The second convergence test shows that the L2 projection seems to have quadratic convergence rate when shortening the distance between nodes: When the distance between nodes is halved, the approximated node function value is four times closer to the actual value.